<a href="https://colab.research.google.com/github/peterchang0414/lecun1989-flax/blob/main/lecun_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1989 Reproduction

In [1]:
!pip install -q flax

     |████████████████████████████████| 184 kB 7.2 MB/s 
     |████████████████████████████████| 136 kB 55.8 MB/s 
     |████████████████████████████████| 70 kB 8.3 MB/s 


In [2]:
import jax
import jax.numpy as jnp
import numpy as np

from flax import linen as nn
from torchvision import datasets

# Adapted from https://github.com/karpathy/lecun1989-repro/blob/master/prepro.py
def get_datasets(n_tr, n_te):
  train_test = {}
  for split in {'train', 'test'}:
    data = datasets.MNIST('./data', train=split=='train', download=True)
    
    n = n_tr if split == 'train' else n_te
    key = jax.random.PRNGKey(42)
    rp = jax.random.permutation(key, len(data))[:n]

    X = jnp.full((n, 16, 16, 1), 0.0, dtype=jnp.float32)
    Y = jnp.full((n, 10), -1.0, dtype=jnp.float32)
    for i, ix in enumerate(rp):
      I, yint = data[int(ix)]
      xi = jnp.array(I, dtype=np.float32) / 127.5 - 1.0
      xi = jax.image.resize(xi, (16, 16), 'bilinear')
      X = X.at[i].set(np.expand_dims(xi, axis=2))
      Y = Y.at[i, yint].set(1.0)
    train_test[split] = (X, Y)
  return train_test

In [3]:
from flax import linen as nn
from flax.training import train_state
from flax.linen.activation import tanh
import optax
from typing import Callable

class Net(nn.Module):
  bias_init: Callable = nn.initializers.zeros
  kernel_init: Callable = nn.initializers.uniform()

  @nn.compact
  def __call__(self, x):
    # For weight initialization, Karpathy used numerator of 2.4 
    # which is very close to sqrt(6) = 2.449... used by he_uniform()
    # By default, weight-sharing forces bias-sharing and therefore
    # we add the bias separately.
    bias1 = self.param('bias1', self.bias_init, (8, 8, 12))
    bias2 = self.param('bias2', self.bias_init, (4, 4, 12))
    bias3 = self.param('bias3', self.bias_init, (30,))
    bias4 = self.param('bias4', nn.initializers.constant(-1.0), (10,))
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x = nn.Conv(features=12, kernel_size=(5,5), strides=2, padding='VALID',
                use_bias=False, kernel_init=self.kernel_init)(x)
    x = tanh(x + bias1)
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x1, x2, x3 = (x[..., 0:8], x[..., 4:12], 
                  jnp.concatenate((x[..., 0:4], x[..., 8:12]), axis=-1))
    slice1 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID', 
                     use_bias=False, kernel_init=self.kernel_init)(x1)
    slice2 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID',
                     use_bias=False, kernel_init=self.kernel_init)(x2)
    slice3 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID',
                     use_bias=False, kernel_init=self.kernel_init)(x3)
    x = jnp.concatenate((slice1, slice2, slice3), axis=-1)
    x = tanh(x + bias2)
    x = x.reshape((x.shape[0], -1))
    x = nn.Dense(features=30, use_bias=False)(x)
    x = tanh(x + bias3)
    x = nn.Dense(features=10, use_bias=False)(x)
    x = tanh(x + bias4)

    return x

In [4]:
@jax.jit
def eval_step(params, X, Y):
  Yhat = Net().apply({'params': params}, X)
  loss = jnp.mean((Yhat - Y)**2)
  err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
  return loss, err

In [5]:
def eval_split(data, split, params):
  X, Y = data[split]
  loss, err = eval_step(params, X, Y)
  print(f"eval: split {split:5s}. loss {loss:e}. error {err*100:.2f}%. misses: {int(err*Y.shape[0])}")

In [6]:
from jax import value_and_grad
import optax
from flax.training import train_state

def create_train_state(key, lr, X):
  model = Net()
  params = model.init(key, X)['params']
  sgd_opt = optax.sgd(lr)
  return train_state.TrainState.create(apply_fn=model.apply, params=params, tx=sgd_opt)

@jax.jit
def train_step(state, X, Y):
  def loss_fn(params):
    Yhat = Net().apply({'params': params}, X)
    loss = jnp.mean((Yhat - Y)**2)
    err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
    return loss, err
  (_, Yhats), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
  state = state.apply_gradients(grads=grads)
  return state

def train_one_epoch(state, X, Y):
  for step_num in range(X.shape[0]):
    x, y = np.expand_dims(X[step_num], 0), np.expand_dims(Y[step_num], 0)
    state = train_step(state, x, y)
  return state

def train(key, data, epochs, lr):
  Xtr, Ytr = data['train']
  Xte, Yte = data['test']
  train_state = create_train_state(key, lr, Xtr)
  for epoch in range(epochs):
    print(f"epoch {epoch+1}")
    train_state = train_one_epoch(train_state, Xtr, Ytr)
    for split in ['train', 'test']:
      eval_split(data, split, train_state.params)


In [ ]:
data = get_datasets(7291, 2007)

In [8]:
key = jax.random.PRNGKey(42)
key, subkey = jax.random.split(key)

train(key, data), 23, 0.03)

KeyboardInterrupt: ignored

Results:

```
epoch 23
eval: split train. loss 6.397844e-03. error 0.97%. misses: 71
eval: split test . loss 2.608742e-02. error 3.84%. misses: 77
```



# "Modern" Adjustments

In [50]:
!pip install -q flax

In [51]:
import jax
import jax.numpy as jnp
import numpy as np

from flax import linen as nn
from torchvision import datasets

# Adapted from https://github.com/karpathy/lecun1989-repro/blob/master/prepro.py
def get_datasets(n_tr, n_te):
  train_test = {}
  for split in {'train', 'test'}:
    data = datasets.MNIST('./data', train=split=='train', download=True)
    
    n = n_tr if split == 'train' else n_te
    key = jax.random.PRNGKey(42)
    rp = jax.random.permutation(key, len(data))[:n]

    X = jnp.full((n, 16, 16, 1), 0.0, dtype=jnp.float32)
    Y = jnp.full((n, 10), 0, dtype=jnp.float32)
    for i, ix in enumerate(rp):
      I, yint = data[int(ix)]
      xi = jnp.array(I, dtype=np.float32) / 127.5 - 1.0
      xi = jax.image.resize(xi, (16, 16), 'bilinear')
      X = X.at[i].set(np.expand_dims(xi, axis=2))
      Y = Y.at[i, yint].set(1.0)
    train_test[split] = (X, Y)
  return train_test

In [52]:
from flax import linen as nn
from flax.training import train_state
from flax.linen.activation import tanh
import optax
from typing import Callable

class Net(nn.Module):
  bias_init: Callable = nn.initializers.zeros
  kernel_init: Callable = nn.initializers.uniform()

  @nn.compact
  def __call__(self, x):
    # For weight initialization, Karpathy used numerator of 2.4 
    # which is very close to sqrt(6) = 2.449... used by he_uniform()
    # By default, weight-sharing forces bias-sharing and therefore
    # we add the bias separately.
    bias1 = self.param('bias1', self.bias_init, (8, 8, 12))
    bias2 = self.param('bias2', self.bias_init, (4, 4, 12))
    bias3 = self.param('bias3', self.bias_init, (30,))
    bias4 = self.param('bias4', nn.initializers.constant(-1.0), (10,))
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x = nn.Conv(features=12, kernel_size=(5,5), strides=2, padding='VALID',
                use_bias=False, kernel_init=self.kernel_init)(x)
    x = tanh(x + bias1)
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x1, x2, x3 = (x[..., 0:8], x[..., 4:12], 
                  jnp.concatenate((x[..., 0:4], x[..., 8:12]), axis=-1))
    slice1 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID', 
                     use_bias=False, kernel_init=self.kernel_init)(x1)
    slice2 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID',
                     use_bias=False, kernel_init=self.kernel_init)(x2)
    slice3 = nn.Conv(features=4, kernel_size=(5,5), strides=2, padding='VALID',
                     use_bias=False, kernel_init=self.kernel_init)(x3)
    x = jnp.concatenate((slice1, slice2, slice3), axis=-1)
    x = tanh(x + bias2)
    x = x.reshape((x.shape[0], -1))
    x = nn.Dense(features=30, use_bias=False)(x)
    x = tanh(x + bias3)
    x = nn.Dense(features=10, use_bias=False)(x)
    x = x + bias4

    return x

In [115]:
from jax import value_and_grad
import optax
from flax.training import train_state

def learning_rate_fn(initial_rate, epochs, steps_per_epoch):
  return optax.linear_schedule(init_value=initial_rate, end_value=initial_rate/3,
                               transition_steps=epochs*steps_per_epoch)

def create_train_state(key, X, lr_fn):
  model = Net()
  params = model.init(key, X)['params']
  sgd_opt = optax.adamw(lr_fn)
  return train_state.TrainState.create(apply_fn=model.apply, params=params, tx=sgd_opt)

@jax.jit
def train_step(state, X, Y):
  def loss_fn(params):
    Yhat = Net().apply({'params': params}, X)
    loss = jnp.mean(optax.softmax_cross_entropy(logits=Yhat, labels=Y))
    err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
    return loss, err
  (_, Yhats), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
  state = state.apply_gradients(grads=grads)
  return state

def train_one_epoch(state, X, Y):
  for step_num in range(X.shape[0]):
    x, y = np.expand_dims(X[step_num], 0), np.expand_dims(Y[step_num], 0)
    state = train_step(state, x, y)
  return state

def train(key, data, epochs, lr):
  Xtr, Ytr = data['train']
  Xte, Yte = data['test']
  lr_fn = learning_rate_fn(lr, epochs, Xtr.shape[0])
  train_state = create_train_state(key, Xtr, lr_fn)
  for epoch in range(epochs):
    print(f"epoch {epoch+1} with learning rate {lr_fn(train_state.step):.6f}")
    train_state = train_one_epoch(train_state, Xtr, Ytr)
    for split in ['train', 'test']:
      eval_split(data, split, train_state.params)


In [113]:
@jax.jit
def eval_step(params, X, Y):
  Yhat = Net().apply({'params': params}, X)
  loss = jnp.mean(optax.softmax_cross_entropy(logits=Yhat, labels=Y))
  err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
  return loss, err

def eval_split(data, split, params):
  X, Y = data[split]
  loss, err = eval_step(params, X, Y)
  print(f"eval: split {split:5s}. loss {loss:e}. error {err*100:.2f}%. misses: {int(err*Y.shape[0])}")

In [93]:
data = get_datasets(7291, 2007)

In [116]:
key = jax.random.PRNGKey(42)
key, subkey = jax.random.split(key)

train(key, data, 46, 3e-4)

epoch 1 with learning rate 0.000300
eval: split train. loss 3.713962e-01. error 10.25%. misses: 747
eval: split test . loss 3.468869e-01. error 8.92%. misses: 179
epoch 2 with learning rate 0.000296
eval: split train. loss 2.619304e-01. error 7.57%. misses: 552
eval: split test . loss 2.562398e-01. error 7.27%. misses: 146
epoch 3 with learning rate 0.000291
eval: split train. loss 2.097210e-01. error 6.13%. misses: 447
eval: split test . loss 2.185351e-01. error 6.48%. misses: 130
epoch 4 with learning rate 0.000287
eval: split train. loss 1.759413e-01. error 5.24%. misses: 382
eval: split test . loss 1.952333e-01. error 5.53%. misses: 111
epoch 5 with learning rate 0.000283
eval: split train. loss 1.515859e-01. error 4.44%. misses: 324
eval: split test . loss 1.795301e-01. error 5.03%. misses: 101
epoch 6 with learning rate 0.000278
eval: split train. loss 1.312660e-01. error 3.88%. misses: 283
eval: split test . loss 1.684355e-01. error 4.83%. misses: 97
epoch 7 with learning rate 0

Change 1: replace tanh on last layer with FC and use softmax. Lower learning rate to 0.01

```
epoch 23
eval: split train. loss 7.162272e-03. error 0.05%. misses: 4
eval: split test . loss 1.687743e-01. error 4.14%. misses: 83

```

Change 2: change from SGD to AdamW with LR 3e-4, double epochs to 46, decay LR to 1e-4 over the course of training.

```
epoch 46
eval: split train. loss 1.890260e-03. error 0.04%. misses: 2
eval: split test . loss 1.953933e-01. error 4.04%. misses: 81
```

Change 3: Introduce data augmentation, e.g. a shift by at most 1 pixel in both x/y directions, and bump up training time to 60 epochs.

```
# 코드로 형식 지정됨
```

Change 4: add dropout at layer H3, shift activation function to relu, and bring up iterations to 80.

```
# 코드로 형식 지정됨
```




